Datasets used: 
- `adult (Adult)` :<br>The UCI Adult dataset contains 14 features, which can be divided into two categories: demographic and income-related.<br> The demographic features include:

    age: continuous.<br>
    fnlwgt : continuous, represents final weight, which is the number of units in the target population that the responding unit represents.<br>
    workclass: categorical, with values 'Private', 'Local-gov', 'Self-emp-not-inc', 'Federal-gov', 'State-gov', 'Self-emp-inc', 'Without-pay'.<br>
    education: categorical, with values '11th', 'HS-grad', 'Assoc-acdm', 'Some-college', '10th', ..., '9th', 'Doctorate', '12th', '1st-4th', 'Preschool'.<br>
    education-num: continuous with values 1 to 16, one number assigned to each label of education feature.<br>
    marital-status: categorical, with values 'Never-married', 'Married-civ-spouse', 'Widowed', 'Separated', 'Divorced', 'Married-spouse-absent', 'Married-AF-spouse'.<br>
    occupation: categorical, with values such as 'Machine-op-inspct', 'Farming-fishing', 'Protective-serv'.<br>
    relationship: categorical, with values 'Own-child', 'Husband', 'Not-in-family', 'Unmarried', 'Wife', 'Other-relative'.<br>
    race: categorical, with values such as white, black, and Asian.<br>
    sex: categorical, with values male and female.<br>
    
    The income-related features include:<br>

    hours-per-week: continuous.<br>
    native-country: categorical, with values such as United-States, Mexico, and Germany.<br>
    capital-gain: continuous, represent the amount of money an individual has gained from the sale of investments such as stocks, bonds, or real estate.<br>
    capital-loss: continuous, represent the amount of money an individual has lost from the sale of investments such as stocks, bonds, or real estate..<br>
    The target feature is the income, which is binary:<br>
    income: categorical, with values less than or equal to 50K and greater than 50K.<br>

Reference Links: https://archive.ics.uci.edu/ml/datasets/adult

In [1]:
import pandas as pd
import numpy as np

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import facts
from facts.clean import clean_dataset
from facts import valid_ifthens_with_coverage_correctness, rules2rulesbyif
from facts.models import customLogisticRegression
from facts.parameters import ParameterProxy
from facts.formatting import recourse_report_reverse, print_recourse_report, print_recourse_report_cumulative, print_recourse_report_KStest_cumulative
from facts.utils import load_rules_by_if

In [2]:
cor_thres = 0.5
cost_budget = 10
top_count = 20
c_inf = 5

# Dataset

In [3]:
urlfile= "https://raw.githubusercontent.com/columbia/fairtest/master/data/adult/adult.csv"

X = pd.read_csv(urlfile)
df = clean_dataset(X, "adult")

age = [val.left for val in df.age.unique()]
age.sort()

df.head()

,age,Workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,"(34.0, 41.0]",State-gov,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,FullTime,United-States,0
1,"(41.0, 50.0]",Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,Married,White,Male,0,0,PartTime,United-States,0
2,"(34.0, 41.0]",Private,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,FullTime,United-States,0
3,"(50.0, 90.0]",Private,7,Married-civ-spouse,Handlers-cleaners,Married,Black,Male,0,0,FullTime,United-States,0
4,"(26.0, 34.0]",Private,13,Married-civ-spouse,Prof-specialty,Married,Black,Female,0,0,FullTime,Cuba,0


In [4]:
y = df['income']
X = df.drop('income', axis=1)

num_features = X._get_numeric_data().columns.to_list()
cate_features = X.select_dtypes(include=['object','category']).columns.to_list()

# Rules

In [5]:
rules_with_atomic_correctness = load_rules_by_if("rulesAdultFairTest_preproc.data")
rules_with_cumulative_correctness = load_rules_by_if("rulesAdultFairTest_cumulative.data")

In [6]:
feature_weights = {"race" : 100 , "sex": 100, "marital-status": 5, "relationship":5, "age": 10, "occupation": 4, "Workclass": 2, "native-country": 4,
                    "hours-per-week":2, "capital-gain": 1, "capital-loss":1, "education-num": 3}
features_with_binary_cost = cate_features
features_with_proportional_cost = num_features


comparators = facts.feature_change_builder(
    X,
    num_cols=features_with_proportional_cost,
    cate_cols=features_with_binary_cost,
    feature_weights=feature_weights,
    num_normalization=True,
    feats_to_normalize = ["capital-gain","capital-loss"]
)
params = ParameterProxy(featureChanges=comparators)

In [7]:
facts.update_costs_cumulative(rules_with_cumulative_correctness, params)

# Old Metrics

## Weighted Average

In [27]:
top_rules, subgroup_costs = facts.select_rules_subset(
    rules_with_atomic_correctness,
    metric = "weighted-average",
    sort_strategy = "abs-diff-decr",
    top_count = top_count,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules",
    ],
    params=params
)

print_recourse_report(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True
)

If age = (50.0, 90.0], capital-gain = 0, capital-loss = 0:
	Protected Subgroup ' Male', 17.54% covered
		Make capital-gain = 15024 with effectiveness 96.94%.
		Make capital-gain = 7298 with effectiveness 70.55%.
		Aggregate cost of the above recourses = -8.06
	Protected Subgroup ' Female', 15.83% covered
		Make capital-gain = 15024 with effectiveness 87.08%.
		Make capital-gain = 7298 with effectiveness 28.72%.
		Aggregate cost of the above recourses = -4.87
	Bias against  Female. Unfairness measure = 3.19.
If age = (50.0, 90.0], capital-gain = 0:
	Protected Subgroup ' Male', 18.00% covered
		Make capital-gain = 15024 with effectiveness 97.01%.
		Make capital-gain = 7298 with effectiveness 71.06%.
		Aggregate cost of the above recourses = -8.10
	Protected Subgroup ' Female', 16.21% covered
		Make capital-gain = 15024 with effectiveness 87.39%.
		Make capital-gain = 7298 with effectiveness 29.53%.
		Aggregate cost of the above recourses = -4.93
	Bias against  Female. Unfairness measure 

## Minimum Cost Above Threshold -- Equal Cost of Effectiveness (Macro)

In [9]:
top_rules, subgroup_costs = facts.select_rules_subset(
    rules_with_atomic_correctness,
    metric="min-above-thr",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cor_threshold=cor_thres,
    filter_sequence = [
        # "remove-contained",
        # "remove-below-thr",        
        # "remove-fair-rules",
        # "keep-only-min-change"
    ],
    params=params
)

print_recourse_report(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True
)

If age = (34.0, 41.0], capital-loss = 0, marital-status =  Never-married, relationship =  Not-in-family:
	Protected Subgroup ' Male', 2.28% covered
		Make age = (41.0, 50.0], marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 48.37%.
		Make age = (50.0, 90.0], marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 43.79%.
		Make marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 41.83%.
		Make age = (41.0, 50.0], capital-loss = 1902, marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 84.97%.
		Aggregate cost of the above recourses = 20.44
	Protected Subgroup ' Female', 2.77% covered
		Make age = (41.0, 50.0], marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 57.39%.
		Make age = (50.0, 90.0], marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 53.04%.
		Make marital-status =  Married-civ-spouse, relationship =  M

## Number of Rules Above Threshold -- Equal Choice for Recourse

In [10]:
top_rules, subgroup_costs = facts.select_rules_subset(
    rules_with_atomic_correctness,
    metric="num-above-thr",
    sort_strategy="generic-sorting-ignore-forall-subgroups-empty",
    top_count=top_count,
    cor_threshold=cor_thres,
    filter_sequence = [
        # "remove-contained",
        # "remove-below-thr",
        # "remove-fair-rules"
    ],
    params=params
)

print_recourse_report(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True
)

If Workclass =  Private, age = (34.0, 41.0], occupation =  Sales:
	Protected Subgroup ' Male', 1.28% covered
		Make age = (41.0, 50.0], occupation =  Exec-managerial with effectiveness 61.63%.
		Make occupation =  Exec-managerial with effectiveness 51.16%.
		Make age = (41.0, 50.0], occupation =  Prof-specialty with effectiveness 60.47%.
		Make age = (50.0, 90.0], occupation =  Exec-managerial with effectiveness 51.16%.
		Make occupation =  Prof-specialty with effectiveness 45.35%.
		Make age = (50.0, 90.0], occupation =  Prof-specialty with effectiveness 51.16%.
		Make age = (41.0, 50.0] with effectiveness 44.19%.
		Make Workclass =  Local-gov, age = (41.0, 50.0], occupation =  Prof-specialty with effectiveness 51.16%.
		Make age = (50.0, 90.0] with effectiveness 20.93%.
		Make age = (41.0, 50.0], occupation =  Craft-repair with effectiveness 0.00%.
		Make occupation =  Craft-repair with effectiveness 0.00%.
		Make Workclass =  Self-emp-inc, age = (50.0, 90.0], occupation =  Exec-mana

# New Metrics

## Fairness of Mean Recourse Cost ($c_\infty$ implementation) -- Equal Mean Recourse

In [11]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="fairness-of-mean-recourse-cinf",
    c_inf=c_inf,
    sort_strategy="generic-sorting",
    top_count=top_count,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules"
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True
)

If capital-loss = 0, education-num = 11, race =  White:
	Protected Subgroup ' Male', 3.75% covered
		Make education-num = 12 with effectiveness 12.95% and counterfactual cost = 3.0.
		Make education-num = 13 with effectiveness 24.11% and counterfactual cost = 6.0.
		Make capital-loss = 1902, education-num = 13 with effectiveness 56.70% and counterfactual cost = 6.44.
		Make education-num = 13, race =  Asian-Pac-Islander with effectiveness 56.70% and counterfactual cost = 106.0.
		Aggregate cost of the above recourses = 3.16
	Protected Subgroup ' Female', 3.73% covered
		Make education-num = 12 with effectiveness 1.59% and counterfactual cost = 3.0.
		Make education-num = 13 with effectiveness 5.56% and counterfactual cost = 6.0.
		Make capital-loss = 1902, education-num = 13 with effectiveness 16.67% and counterfactual cost = 6.44.
		Make education-num = 13, race =  Asian-Pac-Islander with effectiveness 16.67% and counterfactual cost = 106.0.
		Aggregate cost of the above recourses = 2

## Fairness of Mean Recourse Cost (conditional expectation implementation) -- Equal Conditional Mean Recourse

In [12]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="fairness-of-mean-recourse-conditional",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules"
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True
)

If Workclass =  Private, race =  Black, relationship =  Own-child:
	Protected Subgroup ' Male', 1.36% covered
		Make relationship =  Married with effectiveness 1.00% and counterfactual cost = 5.
		Make race =  White, relationship =  Married with effectiveness 1.00% and counterfactual cost = 105.
		Make race =  White, relationship =  Not-in-family with effectiveness 1.00% and counterfactual cost = 105.
		Make race =  Asian-Pac-Islander, relationship =  Married with effectiveness 1.00% and counterfactual cost = 105.
		Make Workclass =  Self-emp-inc, race =  White, relationship =  Married with effectiveness 1.00% and counterfactual cost = 107.
		Make Workclass =  Self-emp-not-inc, race =  White, relationship =  Married with effectiveness 1.00% and counterfactual cost = 107.
		Make Workclass =  Local-gov, race =  White, relationship =  Married with effectiveness 1.00% and counterfactual cost = 107.
		Make Workclass =  Federal-gov, race =  White, relationship =  Married with effectiveness 1

## Fairness of Recourse at Effectiveness Level -- Equal Cost of Effectiveness (Micro)

In [13]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="min-above-corr",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cor_threshold = cor_thres,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules",
        # "keep-cheap-rules-above-thr-cor",
        # "keep-only-min-change",
        
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True
)

If Workclass =  Private, age = (34.0, 41.0], marital-status =  Never-married, relationship =  Not-in-family:
	Protected Subgroup ' Male', 1.76% covered
		Make marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 36.09% and counterfactual cost = 10.
		Make Workclass =  Self-emp-not-inc, marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 36.09% and counterfactual cost = 12.
		Make Workclass =  Local-gov, marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 36.09% and counterfactual cost = 12.
		Make age = (50.0, 90.0], marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 38.35% and counterfactual cost = 20.
		Make age = (41.0, 50.0], marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 41.35% and counterfactual cost = 20.
		Make Workclass =  Self-emp-inc, marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 41.35% and co

## Fairness of Effectiveness -- Equal Effectiveness

In [14]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="total-correctness",
    sort_strategy="generic-sorting-ignore-forall-subgroups-empty",
    top_count=top_count,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules",
        # "keep-only-min-change"
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    correctness_metric=True
)

If Workclass =  Private, age = (26.0, 34.0], native-country =  United-States, occupation =  Machine-op-inspct:
	Protected Subgroup ' Male', 1.91% covered
		Make occupation =  Sales with effectiveness 4.48% and counterfactual cost = 4.
		Make occupation =  Exec-managerial with effectiveness 6.72% and counterfactual cost = 4.
		Make occupation =  Prof-specialty with effectiveness 6.72% and counterfactual cost = 4.
		Make age = (34.0, 41.0], occupation =  Sales with effectiveness 8.21% and counterfactual cost = 14.
		Make age = (41.0, 50.0], occupation =  Craft-repair with effectiveness 8.21% and counterfactual cost = 14.
		Make age = (34.0, 41.0], occupation =  Craft-repair with effectiveness 8.21% and counterfactual cost = 14.
		Make age = (34.0, 41.0], occupation =  Prof-specialty with effectiveness 22.39% and counterfactual cost = 14.
		Make age = (41.0, 50.0], occupation =  Sales with effectiveness 22.39% and counterfactual cost = 14.
		Make age = (34.0, 41.0], occupation =  Exec-man

## Fairness of Recourse (using two-sample Kolmogorov-Smirnov test) -- Fair Effectiveness-Cost Trade-Off

In [15]:
# taken from the other notebooks
affected_pop_sizes = {" Male": 6732, " Female": 4106}

top_rules,unfairness = facts.select_rules_subset_KStest(
    rules_with_cumulative_correctness,
    affected_pop_sizes,
    top_count=top_count
)

print_recourse_report_KStest_cumulative(
    top_rules,
    population_sizes=affected_pop_sizes,
    unfairness = unfairness,
    show_then_costs=True
    # show_cumulative_plots=True
)

If capital-gain = 0, capital-loss = 0, native-country =  United-States:
	Protected Subgroup ' Male', 85.16% covered out of 6732
		Make capital-gain = 5178 with effectiveness 39.24% and counterfactual cost = 0.05.
		Make capital-gain = 7298 with effectiveness 51.20% and counterfactual cost = 0.07.
		Make capital-gain = 7688 with effectiveness 53.31% and counterfactual cost = 0.08.
		Make capital-gain = 15024 with effectiveness 81.94% and counterfactual cost = 0.15.
		Make capital-loss = 1887 with effectiveness 81.94% and counterfactual cost = 0.43.
		Make capital-loss = 1902 with effectiveness 81.94% and counterfactual cost = 0.44.
		Make capital-loss = 1977 with effectiveness 81.94% and counterfactual cost = 0.45.
		Make capital-gain = 99999 with effectiveness 100.00% and counterfactual cost = 1.0.
	Protected Subgroup ' Female', 85.81% covered out of 4106
		Make capital-gain = 5178 with effectiveness 13.59% and counterfactual cost = 0.05.
		Make capital-gain = 7298 with effectiveness 2

## Fairness of Effectiveness at Recourse Budget -- Equal Effectiveness within Budget (Micro)

In [16]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="max-upto-cost",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cost_threshold = cost_budget,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules",
        # "remove-above-thr-cost"
        #"remove-below-thr",
        #"keep-only-min-change",
        
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    correctness_metric = True
)

If Workclass =  Private, age = (50.0, 90.0], capital-loss = 0, occupation =  Sales:
	Protected Subgroup ' Male', 1.09% covered
		Make occupation =  Prof-specialty with effectiveness 35.94% and counterfactual cost = 4.0.
		Make occupation =  Exec-managerial with effectiveness 37.50% and counterfactual cost = 4.0.
		Make Workclass =  Self-emp-inc, occupation =  Exec-managerial with effectiveness 48.44% and counterfactual cost = 6.0.
		Aggregate cost of the above recourses = 0.48
	Protected Subgroup ' Female', 1.61% covered
		Make occupation =  Exec-managerial with effectiveness 4.76% and counterfactual cost = 4.0.
		Make occupation =  Prof-specialty with effectiveness 4.76% and counterfactual cost = 4.0.
		Make Workclass =  Self-emp-inc, occupation =  Exec-managerial with effectiveness 6.35% and counterfactual cost = 6.0.
		Aggregate cost of the above recourses = 0.06
	Bias against  Female. Unfairness measure = 0.421.
If Workclass =  Private, age = (50.0, 90.0], capital-loss = 0, native-

# Gather all metrics in a table (under construction)

In [17]:
from facts.fairness_metrics_aggr import make_table, auto_budget_calculation

In [18]:
rules_with_atomic_correctness.keys() == rules_with_cumulative_correctness.keys()

True

In [19]:
thens1 = {ifc: {sg: (cov, sorted([t for t, _ in thens])) for sg, (cov, thens) in thencs.items()} for ifc, thencs in rules_with_atomic_correctness.items()}
thens2 = {ifc: {sg: (cov, sorted([t for t, _, _ in thens])) for sg, (cov, thens) in thencs.items()} for ifc, thencs in rules_with_cumulative_correctness.items()}

thens1 == thens2

True

In [20]:
rules_with_both = {}
for ifc, all_thens in rules_with_cumulative_correctness.items():
    new_all_thens = {}
    for sg, (cov, thens_cum) in all_thens.items():
        thens_atom = rules_with_atomic_correctness[ifc][sg][1]
        thens_atom_dict = {then: atom_cor for then, atom_cor in thens_atom}
        new_all_thens[sg] = (cov, [(then, thens_atom_dict[then], cumcor) for then, cumcor, _cost in thens_cum])
    rules_with_both[ifc] = new_all_thens

In [21]:
budgets = auto_budget_calculation(rules_with_cumulative_correctness, cor_thres=0.5, percentiles=[0.3, 0.6, 0.9])
budgets

[5.076880768807688, 10.0, 18.0]

In [22]:
df = make_table(
    rules_with_both,
    sensitive_attribute_vals=["Male", "Female"],
    effectiveness_thresholds=[0.3, 0.7],
    cost_budgets=budgets,
    params=params
)

In [23]:
df

subgroup weighted-average  \
                                                subgroup             Male   
0                                       capital-loss = 0        -0.678264   
1                                       capital-gain = 0        -5.905823   
2                     capital-gain = 0, capital-loss = 0        -4.118241   
3      capital-loss = 0, native-country =  United-States        -0.702256   
4      capital-gain = 0, native-country =  United-States        -5.708898   
...                                                  ...              ...   
12875  Workclass =  Private, capital-loss = 0, marita...        -0.002538   
12876  Workclass =  Private, age = (50.0, 90.0], race...        -0.000514   
12877  capital-gain = 0, hours-per-week = OverTime, n...        -0.004076   
12878  capital-gain = 0, capital-loss = 0, hours-per-...        -0.003880   
12879  Workclass =  Private, capital-gain = 0, educat...        -0.004146   

                    (mincost-above-th, 0.3)              \
             Female                    Male      Female   
0     -1.933603e-01                0.453857         inf   
1     -2.941050e+00                0.050131    0.150242   
2     -1.944884e+00                0.050131    0.150242   
3     -1.958153e-01                0.433196         inf   
4     -3.038852e+00                0.051781    0.150242   
...             ...                     ...         ...   
12875 -2.815858e-03              110.433196  110.433196   
12876 -1.092082e-18                     inf         inf   
12877 -3.047360e-03              100.072981  100.072981   
12878 -2.692247e-03              100.072981  100.072981   
12879 -3.346720e-03                     inf         inf   

      (mincost-above-th, 0.7)             (number-above-th, 0.3)         \
                         Male      Female                   Male Female   
0                         inf         inf                     -1      0   
1                    0.150242    0.150242                     -6     -2   
2                    0.150242    0.150242                     -6     -2   
3                         inf         inf                     -3      0   
4                    0.150242    0.150242                     -5     -2   
...                       ...         ...                    ...    ...   
12875                     inf         inf                     -3     -4   
12876                     inf         inf                      0      0   
12877              100.150242  100.150242                     -6     -5   
12878              100.150242  100.150242                     -9     -5   
12879                     inf         inf                      0      0   

      (number-above-th, 0.7)  ... (eff-within-budget, 18.0)            \
                        Male  ...                      Male    Female   
0                          0  ...                  0.305134  0.093235   
1                         -2  ...                  1.000000  1.000000   
2                         -2  ...                  1.000000  1.000000   
3                          0  ...                  0.316954  0.095106   
4                         -2  ...                  1.000000  1.000000   
...                      ...  ...                       ...       ...   
12875                      0  ...                  0.184211  0.134615   
12876                      0  ...                       inf       inf   
12877                     -2  ...                       inf       inf   
12878                     -2  ...                       inf       inf   
12879                      0  ...                  0.093750  0.082353   

      (cost-of-effectiveness, 0.3)             (cost-of-effectiveness, 0.7)  \
                              Male      Female                         Male   
0                         0.453857         inf                          inf   
1                         0.050131    0.150242                     0.150242   
2                         0.050131  

## 2-level index: Examples of usage

In [24]:
df["subgroup"]

,subgroup
0,capital-loss = 0
1,capital-gain = 0
2,"capital-gain = 0, capital-loss = 0"
3,"capital-loss = 0, native-country = United-States"
4,"capital-gain = 0, native-country = United-States"
...,...
12875,"Workclass = Private, capital-loss = 0, marita..."
12876,"Workclass = Private, age = (50.0, 90.0], race..."
12877,"capital-gain = 0, hours-per-week = OverTime, n..."
12878,"capital-gain = 0, capital-loss = 0, hours-per-..."


In [25]:
df["weighted-average", "Male"]

0       -0.299233
1       -0.605355
2       -0.498758
3       -0.309815
4       -0.660429
           ...   
12875   -0.052917
12876   -0.029851
12877   -0.176617
12878   -0.165710
12879   -0.017413
Name: (weighted-average, Male), Length: 12880, dtype: float64

In [26]:
df[("mincost-above-th", 0.3), "Female"]

0        inf
1        1.0
2        1.0
3        inf
4        1.0
        ... 
12875    4.0
12876    inf
12877    2.0
12878    2.0
12879    inf
Name: ((mincost-above-th, 0.3), Female), Length: 12880, dtype: float64

In [27]:
(df[("mincost-above-th", 0.3), "Male"] - df[("mincost-above-th", 0.3), "Female"]).dropna().value_counts()

 0.0    3343
-inf    1288
 inf     385
 1.0     316
-1.0     258
-2.0       5
dtype: int64